In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle
import joblib
import json
import witwidget

from collections import OrderedDict
from sklearn.metrics import accuracy_score


In [ ]:
model_file = open('data/RF19.sav', 'rb')
model = pickle.load(model_file, encoding='latin1')

scaler_file = open('data/scaler.sav', 'rb')
scaler = pickle.load(scaler_file, encoding='latin1')

In [ ]:
features = ['MAC', 'TAC', 'MinNOC_CSF1PO', 'MinNOC_D16S539', 'PercAF_D1S1656', 'AlleleCount_D3S1358', 'AlleleCount_D8S1179', 'MinNOC_Penta D', 'MinNOC_Penta E', 'SumAF_TH01', 'AlleleCount_TPOX', 'MinNOC_TPOX', 'stdHeight_vWA', 'stdAllele', 'MAC0', 'MAC5-6', 'peaksBelowRFU', 'MatchProbability', 'MinNOC']

def gettingData(FEATURES):
    with open("data/Features.txt") as json_file:
        FeaturesSet = json.load(json_file)
    sampleNames = []
    feature_labels = []
    ordered_features = []
    
    for sample, items in FeaturesSet.items():
        sampleList = []
        feature_labels.append(int(items['NOC']))
        sampleNames.append(sample)
        for locus, item in sorted(items["Locus"].items()):
            for feature, featValue in sorted(item.items()):
                if feature in FEATURES:
                    sampleList.append(float(featValue))
                else:
                    continue
        del items['Locus']
        del items['NOC']
        for feature in sorted(items):
            if feature in FEATURES:
                continue
            else:
                del items[feature]
        ordered = OrderedDict(sorted(items.items()))
        newSampleList = sampleList + list(ordered.values()) #locus
        ordered_features.append(list(map(float, newSampleList))) # locus
    featuresOrderedSet = np.array(ordered_features)
    # normalise
    featuresOrderedSet=scaler.transform(featuresOrderedSet)
    return featuresOrderedSet, feature_labels, sampleNames

In [ ]:
featuresOrderedSet, feature_labels, sampleNames = gettingData(features)

print(len(feature_labels))
print(feature_labels)

test_data = pd.DataFrame.from_dict(featuresOrderedSet)
test_data.columns = features
test_data.index = sampleNames
pd.set_option('display.max_columns', None)
test_data

test_labels = feature_labels

In [ ]:
acc = accuracy_score(test_labels, model.predict(test_data))
print(acc)
print(model)

In [ ]:
test_examples = np.hstack((test_data, test_labels))
print(test_data)

print(test_examples)

#config_builder = (WitConfigBuilder(test_examples.tolist()).set_multi_class(True)